# Detection signature : Train & Evaluation :

# 📄 **Détection de Signatures - Entraînement et Évaluation**

---

## 🎯 **Objectif du projet**  
L'objectif initial est de pouvoir **automatiser la reconnaissance de documents signés ou non signés**.  
Cela permettrait :  
✅ D'accélérer le traitement automatique des documents.  
✅ D'améliorer la précision de la détection de signatures.  
✅ De faciliter la vérification d'authenticité des documents.  

---

## 🔎 **Contexte du projet**  
Les signatures manuscrites ne sont **pas des classes** déjà présentes dans les jeux de données utilisés pour l'entraînement initial des modèles de détection d'objets.  
Ainsi, une phase de **fine-tuning** est nécessaire pour ajuster les modèles :  
✔️ Tirer parti des poids déjà pré-entraînés sur des bases d'images générales.  
✔️ Affiner le modèle pour détecter spécifiquement des signatures dans des documents.  
✔️ Améliorer la capacité du modèle à extraire des caractéristiques spécifiques aux signatures.  

---

## 🚀 **Pourquoi le Fine-Tuning est Nécessaire ?**  
Les modèles de détection d'objets (comme YOLO, Faster R-CNN, RetinaNet, etc.) sont généralement pré-entraînés sur des jeux de données comme **COCO** ou **ImageNet**.  
👉 Cependant, ces jeux de données ne contiennent pas de classes spécifiques aux signatures manuscrites.  
👉 Un ajustement est donc nécessaire pour adapter le modèle à ce cas d'usage particulier.  

### ➡️ **Avantages du Fine-Tuning :**  
✅ Accélération du processus d'entraînement grâce aux poids déjà pré-entraînés.  
✅ Extraction de caractéristiques générales des images (formes, textures, motifs).  
✅ Adaptation rapide à un domaine spécifique (reconnaissance de signatures).  
✅ Optimisation plus rapide et plus efficace.  

---

## ⚙️ **Approche Technique**  
### 1. **Préparation des Données :**  
- Extraction des signatures depuis des fichiers PDF.  
- Annotation manuelle des signatures dans les documents (bounding boxes).  
- Augmentation des données pour enrichir le dataset (rotation, recadrage, etc.).  

### 2. **Modèles Utilisés :**  
- 🔹 **YOLOv8** 
- 🔹 **YOLOv11**  


### 3. **Stratégie de Fine-Tuning :**  
✔️ Chargement des poids pré-entraînés.  
✔️ Congélation des premières couches du modèle (pour conserver les caractéristiques générales).  
✔️ Ajustement des couches finales pour apprendre la spécificité des signatures.  
✔️ Entraînement du modèle avec des hyperparamètres adaptés :  
- Taux d'apprentissage (`learning rate`)  
- Taille de batch (`batch size`)  
- Nombre d'époques (`epochs`)  
✔️ Ajustement de la fonction de perte pour une meilleure convergence.  

---

## 📊 **Évaluation des Modèles**  
- **Précision (Precision)** – capacité à ne détecter que des signatures réelles.  
- **Rappel (Recall)** – capacité à détecter toutes les signatures présentes.  
- **mAP (mean Average Precision)** – score moyen global de détection.  
- **F1-Score** – équilibre entre la précision et le rappel.  
- **Taux de faux positifs/négatifs** – capacité à éviter les erreurs de détection.  

---

## 🏆 **Résultats Attendus**  
✅ Détection fiable des signatures dans différents types de documents.  
✅ Réduction du nombre de faux positifs et faux négatifs.  
✅ Optimisation rapide grâce à l'utilisation de poids pré-entraînés.  
✅ Automatisation du processus de validation de documents signés.  

---

## 🚨 **Défis Potentiels**  
⚠️ Mauvaise qualité des signatures (floues, partielles, surimposées).  
⚠️ Variabilité des styles de signature (taille, forme, orientation).  
⚠️ Présence de bruits visuels (tampons, annotations).  
⚠️ Équilibre entre la précision et le rappel.  

---

## ✅ **Prochaines Étapes**  
🔎 Évaluer la performance des modèles après fine-tuning.  
📉 Comparer les résultats des différents modèles.  
🧪 Ajuster les hyperparamètres pour améliorer la convergence.  


In [1]:
%pip install --upgrade supervision roboflow

  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'c:\\Users\\ilyes\\intelligIA\\myenv\\Lib\\site-packages\\cv2\\cv2.pyd'
Check the permissions.



In [2]:
%%capture
!pip install --upgrade wandb

## Import Libraries : 

In [3]:
import os
import importlib
import shutil
import subprocess
import time
import math
import yaml
from typing import Optional
import requests
import pandas as pd
import requests
import json
import gc
import numpy as np
from random import randrange, sample

from abc import ABC, abstractmethod
import dataclasses
from dataclasses import dataclass, asdict, field
from pathlib import Path
from typing import Dict, Any, Optional, List, Type, Union, Callable

from tqdm import tqdm
from pprint import pprint
from datetime import datetime
from dotenv import load_dotenv
import uuid

from IPython.display import Markdown, display, Image
import matplotlib.pyplot as plt

import cv2
import torch
import supervision as sv
import glob

import logging
logger = logging.getLogger("logger")
logging.basicConfig(level=logging.INFO)

# 📂 **Dataset**

---

## **Présentation du Dataset**  
Ce dataset a été développé pour entraîner des modèles de détection de signatures manuscrites dans divers types de documents.  
Il combine des données provenant de plusieurs sources publiques et d'un dataset personnalisé développé par **IntelligIA**.  
Les données ont été soigneusement annotées et unifiées dans un format standardisé **COCO JSON** pour une compatibilité optimale avec les modèles de détection d'objets.  

---

<table>
  <tr>
    <td style="text-align: center; padding: 10px;">
      <a href="https://universe.roboflow.com/tech-ysdkk/signature-detection-hlx8j">
        <img src="https://app.roboflow.com/images/download-dataset-badge.svg">
      </a>
    </td>
    <td style="text-align: center; padding: 10px;">
      <a href="https://huggingface.co/datasets/tech4humans/signature-detection">
        <img src="https://huggingface.co/datasets/huggingface/badges/resolve/main/dataset-on-hf-md.svg" alt="Dataset on HF">
      </a>
    </td>
  </tr>
</table>

---

## **Composantes du Dataset**  
### 🔹 **1. Tobacco800** ([Lien](https://paperswithcode.com/dataset/tobacco-800))  
- Un sous-ensemble du **Complex Document Image Processing (CDIP)** Test Collection.  
- Contient des images scannées de documents liés à l'industrie du tabac.  
- Créé par l'Institut de Technologie de l'Illinois (IIT).  
- **Format :** TIFF converti en COCO JSON.  

---

### 🔹 **2. Signatures-XC8UP** ([Lien](https://universe.roboflow.com/roboflow-100/signatures-xc8up))  
- Fait partie de **Roboflow 100**, une initiative d'Intel.  
- Contient **368 images annotées** pour la détection de signatures manuscrites.  
- Les annotations sont en format **COCO JSON**.  
- **Types de signatures :** Variété de styles et de tailles.  

---

### 🔹 **3. Dataset IntelligIA**  
- Créé et annoté manuellement par **IntelligIA** pour une tâche spécifique de détection de signatures dans des documents professionnels.  
- Inclut **178 images** de documents (format PDF, JPEG et PNG).  
- Les annotations sont en **format COCO JSON**.  
- Les données sont issues de documents réels (assurances, contrats, formulaires).  
- **Types de signatures détectées :**  
    - Signatures manuscrites.  
    - Signatures électroniques.  
    - Signatures partiellement visibles ou incomplètes.  

---

## **Détails du Dataset**  
| Type de Données | Nombre d'Images | % du Total |
|-----------------|------------------|------------|
| **Entraînement** | 1 980 | 70% |
| **Validation**   | 420   | 15% |
| **Test**         | 419   | 15% |
| **Total**        | 2 819 | 100% |

---

## **Format du Dataset**  
✅ **Format :** COCO JSON  
✅ **Taille des fichiers :** 1.2 Go  
✅ **Licence :** Apache 2.0  

---

## 📊 **Pourquoi ces Datasets ?**  
✅ Le dataset **Tobacco800** permet une meilleure reconnaissance de texte complexe dans des documents professionnels.  
✅ Le dataset **Signatures-XC8UP** améliore la reconnaissance de signatures manuscrites.  
✅ Le dataset **IntelligIA** permet une adaptation à des documents réels professionnels (assurances, contrats).  
✅ La diversité des données permet au modèle d'apprendre à détecter des signatures dans différents contextes, styles et qualités d'image.  




## Convert Datasets yolo format to coco json format :

In [ ]:
import os
import json
import cv2
import glob
import numpy as np
import sys
import io


IMG_DIR = 'datasets/signature/train/images/'    
LABEL_DIR = 'datasets/signature/train/labels/'   
OUTPUT_JSON = 'dataset_coco.json'


CATEGORIES = [
    {"id": 0, "name": "signature"}
]


images = []
annotations = []
categories = CATEGORIES
annotation_id = 1


def sanitize_path(path):
    return os.path.normpath(path)


def read_image(img_path):
    try:

        img_path_encoded = os.fsencode(img_path).decode('utf-8')
        image = cv2.imdecode(np.fromfile(img_path_encoded, dtype=np.uint8), cv2.IMREAD_COLOR)
        return image
    except Exception as e:
        print(f"❌ Erreur lors de la lecture de l'image {img_path}: {e}")
        return None


def yolo_to_coco():
    global annotation_id

  
    image_paths = glob.glob(sanitize_path(os.path.join(IMG_DIR, '*.jpg'))) + \
                  glob.glob(sanitize_path(os.path.join(IMG_DIR, '*.png')))

    image_id = 1
    for img_path in image_paths:
        try:
            file_name = os.path.basename(img_path)
            print(f"🔎 Traitement de l'image : {file_name}")

    
            image = read_image(img_path)
            if image is None:
                print(f"❌ Impossible de lire l'image : {img_path}")
                continue

            height, width = image.shape[:2]

            images.append({
                'id': image_id,
                'file_name': file_name,
                'height': height,
                'width': width
            })

            label_path_jpg = sanitize_path(os.path.join(LABEL_DIR, file_name.replace('.jpg', '.txt')))
            label_path_png = sanitize_path(os.path.join(LABEL_DIR, file_name.replace('.png', '.txt')))
            label_path = label_path_jpg if os.path.exists(label_path_jpg) else label_path_png

            if os.path.exists(label_path):
                with open(label_path, 'r', encoding='utf-8') as f:
                    lines = f.readlines()
                    for line in lines:
                        try:
                            class_id, x_center, y_center, w, h = map(float, line.strip().split())

                            x = int((x_center - w / 2) * width)
                            y = int((y_center - h / 2) * height)
                            w = int(w * width)
                            h = int(h * height)

                            annotations.append({
                                'id': annotation_id,
                                'image_id': image_id,
                                'category_id': int(class_id),
                                'bbox': [x, y, w, h],
                                'area': w * h,
                                'iscrowd': 0
                            })

                            annotation_id += 1

                        except Exception as e:
                            print(f"❌ Erreur lors du traitement de l'annotation : {line.strip()} → {e}")

            else:
                print(f"⚠️ Aucun fichier d'annotation trouvé pour l'image : {file_name}")

            image_id += 1

        except Exception as e:
            print(f"❌ Erreur lors du traitement de l'image {file_name} : {e}")


print("🔄 Conversion YOLO → COCO en cours...")
yolo_to_coco()

coco_output = {
    'images': images,
    'annotations': annotations,
    'categories': categories
}

with open(OUTPUT_JSON, 'w', encoding='utf-8') as json_file:
    json.dump(coco_output, json_file, indent=4)

print(f"✅ Fichier COCO JSON sauvegardé : {OUTPUT_JSON}")



🔄 Conversion YOLO → COCO en cours...
🔎 Traitement de l'image : 01 Arr╦åtÔÇÜ de permis de construire_page-0001.jpg
🔎 Traitement de l'image : 01 Arr╦åtÔÇÜ permis de construire_BESSAN_page-0002.jpg
🔎 Traitement de l'image : 01 Arr╦åtÔÇÜ permis de construire_BESSAN_page-0005.jpg
🔎 Traitement de l'image : 01 Arr╦åtÔÇÜ permis de construire_BESSAN_page-0009.jpg
🔎 Traitement de l'image : 01 Arr╦åtÔÇÜ permis de construire_BESSAN_page-0011.jpg
🔎 Traitement de l'image : 01 Arr╦åtÔÇÜ permis de construire_BESSAN_page-0015.jpg
🔎 Traitement de l'image : 01 Arr╦åtÔÇÜ permis de construire_BESSAN_page-0017.jpg
🔎 Traitement de l'image : 01 Arr╦åtÔÇÜ permis de construire_BESSAN_page-0025.jpg
🔎 Traitement de l'image : 01. Arr╦åtÔÇÜ de permis de construire Arles_page-0003.jpg
🔎 Traitement de l'image : 01. Arr╦åtÔÇÜ de permis de construire Arles_page-0008.jpg
🔎 Traitement de l'image : 01. Arr╦åtÔÇÜ de permis de construire Arles_page-0011.jpg
🔎 Traitement de l'image : 01. Arr╦åtÔÇÜ de permis de construire Arl

## Display dataset sample

In [3]:
import os
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

IMG_DIR = 'datasets/signature/train/images'
LABEL_DIR = 'datasets/signature/train/labels'
GRID_SIZE = 6

CLASSES = ["signature"]

def load_yolo_annotations(label_path, width, height):
    annotations = []
    if os.path.exists(label_path):
        with open(label_path, 'r', encoding='utf-8') as f:  
            lines = f.readlines()
            for line in lines:
                try:
                    class_id, x_center, y_center, w, h = map(float, line.strip().split())
                    x = int((x_center - w / 2) * width)
                    y = int((y_center - h / 2) * height)
                    w = int(w * width)
                    h = int(h * height)
                    annotations.append({
                        'class_id': int(class_id),
                        'bbox': [x, y, w, h]
                    })
                except Exception as e:
                    print(f"❌ Erreur lors de la lecture de l'annotation : {e}")
    return annotations

def annotate_image(image, annotations):
    for ann in annotations:
        x, y, w, h = ann['bbox']
        class_id = ann['class_id']
        label = CLASSES[class_id]
        
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(
            image, 
            label, 
            (x, y - 10), 
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.6, 
            (0, 255, 0), 
            2
        )
    return image

def load_and_annotate_images():
    images = []
    
    image_paths = glob.glob(os.path.join(IMG_DIR, '*.jpg')) + glob.glob(os.path.join(IMG_DIR, '*.png'))
    
    image_paths = [path.replace("\\", "/") for path in image_paths]
    
    for img_path in image_paths[:GRID_SIZE * GRID_SIZE]:
        try:
            file_name = os.path.basename(img_path)
            label_path_jpg = os.path.join(LABEL_DIR, file_name.replace('.jpg', '.txt'))
            label_path_png = os.path.join(LABEL_DIR, file_name.replace('.png', '.txt'))
            label_path = label_path_jpg if os.path.exists(label_path_jpg) else label_path_png

            if not os.path.exists(img_path):
                print(f"❌ Fichier introuvable : {img_path}")
                continue

           
            image = cv2.imdecode(np.fromfile(img_path, dtype=np.uint8), cv2.IMREAD_COLOR)
            if image is None:
                print(f"❌ Erreur lors du chargement de l'image : {img_path}")
                continue

            height, width = image.shape[:2]

           
            annotations = load_yolo_annotations(label_path, width, height)

           
            annotated_image = annotate_image(image.copy(), annotations)

    
            annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
            images.append(annotated_image)

        except Exception as e:
            print(f"❌ Erreur lors du traitement de l'image {img_path} : {e}")

    return images

def display_images_in_grid(images, grid_size=(GRID_SIZE, GRID_SIZE), fig_size=(15, 15)):
    fig, axes = plt.subplots(grid_size[0], grid_size[1], figsize=fig_size)
    axes = axes.flatten()
    
    for i in range(len(axes)):
        if i < len(images):
            axes[i].imshow(images[i])
            axes[i].axis('off')
        else:
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()


print("🔄 Chargement des images avec annotations...")
annotated_images = load_and_annotate_images()

if annotated_images:
    print(f"✅ {len(annotated_images)} images chargées et annotées.")
    display_images_in_grid(annotated_images)
else:
    print("⚠️ Aucune image chargée.")


🔄 Chargement des images avec annotations...


✅ 36 images chargées et annotées.


<Figure size 1500x1500 with 36 Axes>

## YOLO : 

In [24]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.86  Python-3.12.3 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
Setup complete  (8 CPUs, 15.8 GB RAM, 270.5/284.9 GB disk)


In [27]:
from ultralytics import settings

# Update a setting
settings.update({
    'mlflow': False,
    'clearml': False,
    'comet': False,
    'dvc': False,
    'hub': False,
    'neptune': False,
    'raytune': False,
    'tensorboard': False,
    'wandb': True
})

## Fine tuning Modele Yolo :

In [ ]:

best_model_path = os.path.join(
    "runs", "detect", "YOLOv8_finetune_signature", "weights", "best.pt"
)
save_as = "finetuned_yolov8_signature_best.pt"

if os.path.exists(best_model_path):
    shutil.copy(best_model_path, save_as)
    print(f"✅ Meilleur modèle sauvegardé sous : {save_as}")
else:
    print(f"⚠️ Aucun fichier best.pt trouvé dans : {best_model_path}")



In [2]:
import os
import sys
import logging

os.environ["WANDB_DISABLED"] = "true"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

logging.getLogger().setLevel(logging.WARNING)

import mlflow
from ultralytics import YOLO

MODEL_CHECKPOINT = "Assurances/YOLO_Trained/yolov8sFinal.pt"
DATA_YAML = "C:/Users/ilyes/intelligIA/datasets/signature/signature.yaml"
EPOCHS = 50
BATCH_SIZE = 8
BASE_LR = 5e-5

MLFLOW_TRACKING_URI = "file:///" + os.path.abspath("mlruns")
EXPERIMENT_NAME = "YOLO_Signature_FineTune"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

try:
    with mlflow.start_run() as run:
        mlflow.log_param("model_checkpoint", MODEL_CHECKPOINT)
        mlflow.log_param("data_yaml", DATA_YAML)
        mlflow.log_param("epochs", EPOCHS)
        mlflow.log_param("batch_size", BATCH_SIZE)
        mlflow.log_param("base_lr", BASE_LR)

   
        model = YOLO(MODEL_CHECKPOINT)

      
        results = model.train(
            data=DATA_YAML,
            epochs=EPOCHS,
            batch=BATCH_SIZE,
            lr0=BASE_LR,
            imgsz=640,
            single_cls=True,
            cos_lr=True,
            patience=8,
            freeze=0,
            degrees=10,
            shear=4.0,
            hsv_v=0.1,
            flipud=0.0,
            fliplr=0.5,
            scale=0.5,
            project="Signature-Detection-FineTune",
            name="YOLOv8_finetune_signature",
            verbose=False,
            save=True         
        )

        results_test = model.val(
            data=DATA_YAML,
            split="test",
            imgsz=640,
            conf=0.25
        )
        print("[INFO] Résultats test :", results_test)

        try:
            mAP50 = results_test.box.maps[0]
            mlflow.log_metric("mAP50_test", mAP50)
        except Exception as e:
            print(f"[WARNING] Impossible de logguer les métriques YOLO : {e}")

  
        export_name = "signature_finetuned"
        export_path = model.export(
            format="torchscript",
            imgsz=640,
            name=export_name,
            dynamic=False,     
            simplify=False     
        )


        print(f"[INFO] Modèle exporté en TorchScript : {export_path}")

      
        if os.path.exists(export_path):
            mlflow.log_artifact(export_path, artifact_path="model")
            print(f"[INFO] Modèle .pt loggué dans MLflow : {export_path}")
        else:
            print("[WARNING] Export .pt introuvable.")

        print("[INFO] Fin du run MLflow :", run.info.run_id)

except Exception as e:
    print("Une erreur est survenue :", e)


New https://pypi.org/project/ultralytics/8.3.91 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.88  Python-3.12.3 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
engine\trainer: task=detect, mode=train, model=Assurances/YOLO_Trained/yolov8sFinal.pt, data=C:/Users/ilyes/intelligIA/datasets/signature/signature.yaml, epochs=50, time=None, patience=8, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=Signature-Detection-FineTune, name=YOLOv8_finetune_signature15, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=0, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visual

train: Scanning C:\Users\ilyes\intelligIA\datasets\signature\train\labels.cache... 143 images, 0 backgrounds, 0 corrupt: 100%|██████████| 143/143 [00:00<?, ?it/s]Scanning C:\Users\ilyes\intelligIA\datasets\signature\train\labels.cache... 143 images, 0 backgrounds, 0 corrupt: 100%|██████████| 143/143 [00:00<?, ?it/s]
val: Scanning C:\Users\ilyes\intelligIA\datasets\signature\valid\labels.cache... 35 images, 0 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<?, ?it/s]Scanning C:\Users\ilyes\intelligIA\datasets\signature\valid\labels.cache... 35 images, 0 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<?, ?it/s]


Plotting labels to Signature-Detection-FineTune\YOLOv8_finetune_signature15\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=5e-05' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to Signature-Detection-FineTune\YOLOv8_finetune_signature15
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      1.539       1.11      1.822         25        640: 100%|██████████| 18/18 [02:11<00:00,  7.32s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:06<00:00,  2.19s/it]


                   all         35         35      0.832      0.943      0.936      0.662

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         35         35      0.974          1      0.995      0.718
EarlyStopping: Training stopped early as no improvement observed in last 8 epochs. Best results observed at epoch 14, best model saved as best.pt.
To update EarlyStopping(patience=8) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

22 epochs completed in 0.677 hours.
Optimizer stripped from Signature-Detection-FineTune\YOLOv8_finetune_signature15\weights\last.pt, 22.5MB
Optimizer stripped from Signature-Detection-FineTune\YOLOv8_finetune_signature15\weights\best.pt, 22.5MB

Validating Signature-Detection-FineTune\YOLOv8_finetune_signature15\weights\best.pt...
Ultralytics 8.3.88  Python-3.12.3 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:05<00:00,  1.81s/it]


                   all         35         35      0.996          1      0.995      0.827
Speed: 2.5ms preprocess, 120.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to Signature-Detection-FineTune\YOLOv8_finetune_signature15


lr/pg0,▁▃▄▆▇█████▇▇▇▇▇▆▆▆▆▅▅▅
lr/pg1,▁▃▄▆▇█████▇▇▇▇▇▆▆▆▆▅▅▅
lr/pg2,▁▃▄▆▇█████▇▇▇▇▇▆▆▆▆▅▅▅
metrics/mAP50(B),▁▄▇▇▇██▆▁██████████▇██
metrics/mAP50-95(B),▃▆▅▄▆▄▄▁▂▅▄▆▇█▆▅▂▅▅▆▄█
metrics/precision(B),▁▄▇█▆██▅▅▇██████▇▆▆▇██
metrics/recall(B),▆▆▆▆███▅▁█▅▇█████▇█▇██
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▅▅▅▅▃▃▄▃▄▃▂▂▂▃▃▃▂▂▁▂▂


Ultralytics 8.3.88  Python-3.12.3 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
Une erreur est survenue : val: Error loading data from C:\Users\ilyes\intelligIA\datasets\signature\test\images
See https://docs.ultralytics.com/datasets for dataset formatting guidance.


In [1]:
import onnx
onnx.__version__


'1.17.0'

In [5]:
from ultralytics import YOLO


MODEL_PT = "Signature-Detection-FineTune\YOLOv8_finetune_signature7\weights\llest.pt" 


model = YOLO(MODEL_PT)

onnx_path = model.export(
    format="onnx",
    opset=12,
    imgsz=640,
    dynamic=False,
    name="signature_finetuned_onnx"
)

print(f"Modèle ONNX généré : {onnx_path}")


Ultralytics 8.3.88  Python-3.12.3 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)


<>:4: SyntaxWarning: invalid escape sequence '\Y'
<>:4: SyntaxWarning: invalid escape sequence '\Y'
C:\Users\ilyes\AppData\Local\Temp\ipykernel_31364\4003746343.py:4: SyntaxWarning: invalid escape sequence '\Y'
  MODEL_PT = "Signature-Detection-FineTune\YOLOv8_finetune_signature7\weights\llest.pt"


Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from 'Signature-Detection-FineTune\YOLOv8_finetune_signature7\weights\llest.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (21.5 MB)

ONNX: starting export with onnx 1.17.0 opset 12...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success  2.4s, saved as 'Signature-Detection-FineTune\YOLOv8_finetune_signature7\weights\llest.onnx' (42.7 MB)

Export complete (3.0s)
Results saved to C:\Users\ilyes\intelligIA\Signature-Detection-FineTune\YOLOv8_finetune_signature7\weights
Predict:         yolo predict task=detect model=Signature-Detection-FineTune\YOLOv8_finetune_signature7\weights\llest.onnx imgsz=640  
Validate:        yolo val task=detect model=Signature-Detection-FineTune\YOLOv8_finetune_signature7\weights\llest.onnx imgsz=640 data=C:/Users/ilyes/intelligIA/datasets/signature/signature.yaml  
Visualize:       https://netron.app
Modèle ONNX généré : Sig